In [2]:
from upsetplot import UpSet
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce

In [3]:
path_manifest = "E:/YandexDisk/Work/bbd/epi_pred/manifests/Illumina"

manifest_450 = pd.read_csv(f"{path_manifest}/humanmethylation450_15017482_v1-2.csv")
manifest_epic = pd.read_csv(f"{path_manifest}/infinium-methylationepic-v-1-0-b5-manifest-file.csv")
manifest_epicv2 = pd.read_csv(f"{path_manifest}/EPIC-8v2-0_A2.csv")
manifest_msa = pd.read_csv(f"{path_manifest}/MSA-48v1-0_20102838_A1.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_5172\2912620775.py:3: DtypeWarning: Columns (11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  manifest_450 = pd.read_csv(f"{path_manifest}/humanmethylation450_15017482_v1-2.csv")
C:\Users\user\AppData\Local\Temp\ipykernel_5172\2912620775.py:4: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  manifest_epic = pd.read_csv(f"{path_manifest}/infinium-methylationepic-v-1-0-b5-manifest-file.csv")
C:\Users\user\AppData\Local\Temp\ipykernel_5172\2912620775.py:5: DtypeWarning: Columns (5,6,7,8,15,23,24,25,26,27,28,29,30,31,32,33,34,36,37,38,39,40,41,42,43,44,45,46,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  manifest_epicv2 = pd.read_csv(f"{path_manifest}/EPIC-8v2-0_A2.csv")
C:\Users\user\AppData\Local\Temp\ipykernel_5172\2912620775.py:6: DtypeWarning: Columns (5,6,7,8,16,26,27,31,32,33,34,37,40,41,42,43,44,45,49) have mixe

In [4]:
cpgs_450 = list(manifest_450['Name'])
cpgs_epic = list(manifest_epic['Name'])
cpgs_epicv2 = list(manifest_epicv2['Name'])
cpgs_msa = list(manifest_msa['Name'])

In [8]:
indices = [
    pd.Index(cpgs_450),
    pd.Index(cpgs_epic),
    pd.Index(cpgs_epicv2),
    pd.Index(cpgs_msa),
]
cpgs_common = reduce(lambda x, y: x.intersection(y), indices)
pd.DataFrame(index=cpgs_common).to_csv(f"{path_manifest}/intersection.csv", index_label='CpGs')

In [ ]:
cpgs_dict = {"450k": cpgs_450, "EPIC": cpgs_epic, "EPICv2": cpgs_epicv2, "MSA": cpgs_msa}
upset_all = list(set().union(*list(cpgs_dict.values())))
df_upset = pd.DataFrame(index=upset_all)
for k, v in cpgs_dict.items():
    df_upset[k] = df_upset.index.isin(v)
df_upset = df_upset.set_index(list(cpgs_dict.keys()))

tmp = plt.figure(figsize=(30, 10))
upset_fig = UpSet(
    df_upset,
    sort_categories_by='input',
    subset_size='count',
    show_counts=True,
    min_degree=0,
    element_size=None,
    totals_plot_elements=3,
    include_empty_subsets=False
)
upset_fig.plot(tmp)
plt.savefig(f"{path_manifest}/upset.png", bbox_inches='tight')
plt.savefig(f"{path_manifest}/upset.pdf", bbox_inches='tight')
plt.close()